## El objetivo de este ejercicio es calcular la función de valor de una política dada usando Diferencias Temporales.
<img src="Prediccion_TD.PNG">

In [26]:
%matplotlib inline

import gym
# import matplotlib
import numpy as np

from collections import defaultdict
# import itertools

# en caso de correrlo en google colab
# de esta manera podremos tener la carpeta lib (donde se encuentra en ambiente Gridworld)

# import sys
#if "../" not in sys.path:
#  sys.path.append("../") 

# !git clone https://github.com/julianfm7/cursoRL-FIUBA

# necesario en google colab para que sys.path busque
# y encuentre la carpeta lib donde se encuentra el ambiente Gridworld

# !mv cursoRL-FIUBA cursoRLFIUBA

# import numpy as np
# from lib.envs.blackjack import BlackjackEnv
from lib.envs.gridworld import GridworldEnv
from lib import plotting

# matplotlib.style.use('ggplot')

from tqdm import tqdm

In [3]:
env = GridworldEnv()
# env = BlackjackEnv()

In [4]:
def td_prediction(policy, env, num_episodes, discount_factor=1.0, alpha=0.01):
    """
    Algoritmo de predicción por diferencias temporales.
    Calcula la función de valor dada una política utilizando diferencias temporales para los updates.
    
    Argumentos:
        policy: Una función que dada una observación toma una acción
        env: ambiente de OpenAI gym.
        num_episodes: número de episodios para samplear en total.
        discount_factor: factor de descuento gama.
    
    Returns:
    Retorna:
        Un diccionario que mapea de estado -> valor
    """
    # la función de valor final
    V = defaultdict(float)
    
    # por cada episodio desde 1 hasta el número de episodios
    for _ in tqdm(range(num_episodes)):
        # hacer print del número de episodio si es múltiplo de 1000 para observar el avance
        
        # resetear el ambiente, obtener el primer estado
        observation = env.reset()
        done = False
        # mientras no lleguemos estado terminal
        while not done:
            # obtener la próxima acción según la política
            accion = policy(observation)
            # tomar la acción en el ambiente
            observacion_nueva, reward, done, _ = env.step(accion)
            # actualizar V según la fórmula de diferencias temporales
            V[(observation)] += alpha*(reward + discount_factor*V[(observacion_nueva)] - V[(observation)])
            # pasar al próximo estado
            observation = observacion_nueva
            
    # devolver la función de valor
    return V    

In [5]:
def sample_policy(observation):
    """
    Política aleatoria
    """
    return np.random.choice(4)

In [6]:
V = td_prediction(sample_policy, env, num_episodes=500000)
# V = td_prediction(sample_policy, env, num_episodes=50000)

100%|██████████| 500000/500000 [05:09<00:00, 1615.85it/s]


In [28]:
# print(V)
Varray = np.zeros(16)
for key, value in V.items():
    Varray[key]=value
print(Varray)

[  0.         -14.12812735 -19.99847639 -21.68445102 -12.37684103
 -17.89439076 -19.5461144  -19.78000921 -19.27260822 -19.30748438
 -17.47910186 -13.50079201 -21.33066319 -19.51006199 -11.66938818
   0.        ]


In [19]:
# Test de la función de valor
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
# np.testing.assert_array_almost_equal(Varray, expected_v, decimal=0)
Varray-expected_v

array([ 0.00000000e+00, -1.28127348e-01,  1.52360796e-03,  3.15548977e-01,
        1.62315897e+00,  1.05609240e-01,  4.53885596e-01,  2.19990788e-01,
        7.27391782e-01,  6.92515625e-01,  5.20898136e-01,  4.99207990e-01,
        6.69336807e-01,  4.89938011e-01,  2.33061182e+00,  0.00000000e+00])